In [14]:
from capella2polarion.connectors import polarion_worker
from capella2polarion.converters import element_converter, converter_config, data_session
import dotenv
import os
import capellambse

In [2]:
dotenv.load_dotenv()
model = capellambse.MelodyModel(os.environ.get("MODEL_PATH"))
worker = polarion_worker.CapellaPolarionWorker(
    polarion_worker.PolarionWorkerParams(
            os.environ.get("POLARION_PROJECT"),
            os.environ.get("POLARION_HOST"),
            os.environ.get("POLARION_PAT"),
            False,
        )
)

In [3]:
worker.load_polarion_work_item_map()

In this example we want to test a jinja template for classes. We want to adjust nothing but the description field and we want to update all classes related to the class, too.

In [70]:
def add_class_incl_dependencies(cls, classes):
    if cls in classes:
        return
    classes.append(cls)
    if cls.super:
        add_class_incl_dependencies(cls.super, classes)
    for property in cls.properties:
        _type = None
        if "type" in property.__dir__():
            _type = property.type
        elif "abstract_type" in property.__dir__():
            _type = property.abstract_type
        if _type and _type.xtype == "org.polarsys.capella.core.data.information:Class":
            add_class_incl_dependencies(_type, classes)

In [72]:
classes = []
add_class_incl_dependencies(model.by_uuid("3e92ba92-89b1-4c4a-abaf-fa78fe95142e"), classes)
len(classes)

36

In [73]:
class_config = converter_config.CapellaTypeConfig(
    "class",
    {
        "jinja_as_description": {
            "template_folder": "element_templates",
            "template_path": "class.html.j2",
        }
    },
    [],
)
serializer = element_converter.CapellaWorkItemSerializer(
    model,
    worker.polarion_data_repo,
    {
        cls.uuid: data_session.ConverterData(
            "pa",
            class_config,
            cls,
        )
        for cls in classes
    },
    False,
)


In [ ]:
wis=serializer.serialize_all()

In [ ]:
worker.client.project_client.work_items.update(wis)